In [1]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import matplotlib.pyplot as plt
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np


In [13]:
from shapely import geometry as gmty

In [60]:
from geofeather import to_geofeather, from_geofeather

In [2]:
shapefile = "/home/mlopez/EXEC/Shapefiles/territoire_guide.shp"

In [8]:
#path for 11 models
path = "/home/mlopez/EXEC/Tg_annual_11_models/"

# Convert all lat lon of 11 models into points 

In [10]:
dfpoints = pcdf.data_to_points(path)

In [11]:
dfpoints.head()

,lat,lon,geometry
0,66.623306,-89.045212,POINT (-89.04521 66.62331)
1,66.623306,-88.961884,POINT (-88.96188 66.62331)
2,66.623306,-88.878548,POINT (-88.87855 66.62331)
3,66.623306,-88.795219,POINT (-88.79522 66.62331)
4,66.623306,-88.711891,POINT (-88.71189 66.62331)


# Create polygons

In [12]:
arr = dfpoints.lon.unique()
arr.sort()
dlon = np.diff(arr)[0] / 2
arr = dfpoints.lat.unique()
arr.sort()
dlat = np.diff(arr)[0] / 2
(dlon, dlat)

(0.04166412353515625, 0.04166603088378906)

In [16]:
dfpoints.geometry = [gmty.Polygon([[lon-dlon, lat-dlat], [lon+dlon, lat-dlat], [lon+dlon, lat+dlat], [lon-dlon, lat+dlat]]) 
                for (lat, lon) in zip(dfpoints.lat, dfpoints.lon)]

In [17]:
dfpoints.geometry[0]

0    POLYGON ((-89.08688 66.58164, -89.00355 66.581...
0    POLYGON ((-140.99999 83.41666, -140.91666 83.4...
Name: geometry, dtype: geometry

In [18]:
dfpoints.head()

,lat,lon,geometry
0,66.623306,-89.045212,"POLYGON ((-89.08688 66.58164, -89.00355 66.581..."
1,66.623306,-88.961884,"POLYGON ((-89.00355 66.58164, -88.92022 66.581..."
2,66.623306,-88.878548,"POLYGON ((-88.92021 66.58164, -88.83688 66.581..."
3,66.623306,-88.795219,"POLYGON ((-88.83688 66.58164, -88.75356 66.581..."
4,66.623306,-88.711891,"POLYGON ((-88.75356 66.58164, -88.67023 66.581..."


In [45]:
%time  dfpolyshape = pcdf.make_grid(dfpoints, shapefile) #10 min with overlay #1miin with sjoin

CPU times: user 9min 41s, sys: 12 s, total: 9min 53s
Wall time: 9min 35s


In [46]:
dfpolyshape.head()

,lat,lon,TER_GUIDE,geometry
0,52.957191,-67.712730,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
1,52.957191,-67.629402,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."
2,52.957191,-67.546066,6opqr,"POLYGON ((-67.56730 52.91553, -67.57974 52.915..."
3,52.873859,-67.796059,6opqr,"POLYGON ((-67.75439 52.89329, -67.75439 52.832..."
4,52.873859,-67.712730,6opqr,"POLYGON ((-67.75439 52.83219, -67.75439 52.893..."


In [61]:
to_geofeather(dfpolyshape, 'Grid-TerritoiresGuides.feather')

In [47]:
OneA =  dfpolyshape[dfpolyshape.TER_GUIDE == "1a"]
OneA

,lat,lon,TER_GUIDE,geometry
32188,46.290791,-72.879189,1a,"POLYGON ((-72.83752 46.26212, -72.83752 46.249..."
32189,46.290791,-72.795860,1a,"POLYGON ((-72.83752 46.24912, -72.83752 46.262..."
32190,46.290791,-72.712532,1a,"POLYGON ((-72.75420 46.24912, -72.75420 46.280..."
32191,46.290791,-72.629204,1a,"POLYGON ((-72.67087 46.24912, -72.67087 46.279..."
32192,46.207458,-73.045853,1a,"MULTIPOLYGON (((-73.00419 46.16711, -73.00419 ..."
...,...,...,...,...
32862,44.958332,-74.208336,1a,"POLYGON ((-74.25000 44.99143, -74.25000 45.000..."
32863,44.958332,-74.125000,1a,"POLYGON ((-74.16666 44.99141, -74.16666 45.000..."
32864,44.958332,-74.041672,1a,"POLYGON ((-74.08334 44.99371, -74.08334 45.000..."
32865,44.958332,-73.958336,1a,"POLYGON ((-74.00000 44.99668, -74.00000 45.000..."


# Create time periods to merge grid with data

## Opening and transfering data into a pandas dataframe

In [29]:
df = pcdf.load_as_df("/home/mlopez/EXEC/Processed Data Varibales/NorESM1-M_rcp45_tg_mean_annual.nc")

/exec/mlopez/Code/GitHub/processing_netcdf.py:23: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()


In [30]:
# Converting degrees in Celsius
dfC = df.copy()
dfC["tg_mean"] = dfC["tg_mean"] -273.15
dfC.head()

tg_mean
lat       lon        time                 
66.623306 -89.045212 1950-01-01 -12.665070
                     1951-01-01 -14.071350
                     1952-01-01 -14.472076
                     1953-01-01 -15.160706
                     1954-01-01 -14.090820

In [31]:
# Defining periods and grouping by time and periods to create maps
year_groups = {y:0 for y in range(1980,2011)}
year_groups.update({y:1 for y in range(2041,2071)})
year_groups.update({y:2 for y in range(2071,2101)})
dfC2 = dfC.reset_index()
dfp = dfC2.groupby([dfC2.time.dt.year.map(year_groups), "lat","lon"]).mean()
dfp

tg_mean
time lat       lon                  
0.0  40.041039 -89.045212  11.474038
               -88.961884  11.452959
               -88.878548  11.440704
               -88.795219  11.419402
               -88.711891  11.434692
...                              ...
2.0  66.623306 -61.629639  -6.771683
               -61.546310  -7.084950
               -61.462978  -6.876992
               -61.379650  -6.807693
               -61.296322  -6.205523

[228618 rows x 1 columns]

# Historic Period

In [32]:
df_h = dfp.query("time==0.0")
df_h.head()

tg_mean
time lat       lon                  
0.0  40.041039 -89.045212  11.474038
               -88.961884  11.452959
               -88.878548  11.440704
               -88.795219  11.419402
               -88.711891  11.434692

## Processing to merge data with mask

In [34]:
df_h = df_h.reset_index()
df_h.set_index(["lat","lon"])

time    tg_mean
lat       lon                        
40.041039 -89.045212   0.0  11.474038
          -88.961884   0.0  11.452959
          -88.878548   0.0  11.440704
          -88.795219   0.0  11.419402
          -88.711891   0.0  11.434692
...                    ...        ...
66.623306 -61.629639   0.0  -9.656209
          -61.546310   0.0  -9.948932
          -61.462978   0.0  -9.735876
          -61.379650   0.0  -9.655593
          -61.296322   0.0  -9.051710

[76206 rows x 2 columns]

In [70]:
#For one subregion
#OneA.set_index(["lat","lon"])

In [62]:
#For all subregions
dfpolyshape.set_index(["lat","lon"])

TER_GUIDE  \
lat       lon                    
52.957191 -67.712730     6opqr   
          -67.629402     6opqr   
          -67.546066     6opqr   
52.873859 -67.796059     6opqr   
          -67.712730     6opqr   
...                        ...   
45.041668 -72.208336        2c   
          -72.125000        2c   
          -72.041672        2c   
          -71.958336        2c   
          -71.875000        2c   

                                                               geometry  
lat       lon                                                            
52.957191 -67.712730  POLYGON ((-67.67107 52.93424, -67.67107 52.915...  
          -67.629402  POLYGON ((-67.67107 52.91553, -67.67107 52.934...  
          -67.546066  POLYGON ((-67.56730 52.91553, -67.57974 52.915...  
52.873859 -67.796059  POLYGON ((-67.75439 52.89329, -67.75439 52.832...  
          -67.712730  POLYGON ((-67.75439 52.83219, -67.75439 52.893...  
...                                                                 ...  
45.041668 -72.208336  POLYGON ((-72.25000 45.00450, -72.25000 45.083...  
          -72.125000  POLYGON ((-72.16666 45.00508, -72.16666 45.083...  
          -72.041672  POLYGON ((-72.08334 45.00564, -72.08334 45.083...  
          -71.958336  POLYGON ((-72.00000 45.00667, -72.00000 45.083...  
          -71.875000  POLYGON ((-71.91666 45.04812, -71.91666 45.083...  

[33198 rows x 2 columns]

## Merge data with mask 

In [49]:
#for one region
#dfTG2 = pd.merge(df_h, OneA, on=["lat","lon"])

In [65]:
dfTGall = pd.merge(df_h, dfpolyshape, on=["lat","lon"])

In [66]:
dfTGall

,time,lat,lon,tg_mean,TER_GUIDE,geometry
0,0.0,44.957512,-74.629120,7.079183,1a,"POLYGON ((-74.62590 44.99918, -74.58746 44.999..."
1,0.0,44.957512,-74.545792,7.053729,1a,"POLYGON ((-74.58746 44.99876, -74.58746 44.999..."
2,0.0,44.957512,-74.462463,7.030324,1a,"POLYGON ((-74.50413 44.99784, -74.50413 44.999..."
3,0.0,44.957512,-74.379128,7.015834,1a,"POLYGON ((-74.42079 44.99530, -74.42079 44.999..."
4,0.0,44.957512,-74.295799,6.978581,1a,"POLYGON ((-74.33746 44.99215, -74.33746 44.999..."
...,...,...,...,...,...,...
16563,0.0,52.873859,-67.629402,-3.216939,6opqr,"POLYGON ((-67.67107 52.83219, -67.67107 52.915..."
16564,0.0,52.873859,-67.546066,-3.166103,6opqr,"POLYGON ((-67.58773 52.83219, -67.58773 52.904..."
16565,0.0,52.957191,-67.712730,-3.351753,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
16566,0.0,52.957191,-67.629402,-3.251923,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."


In [71]:
#dfTG2

In [72]:
#dfTG2.geometry

# ---- Create GeoJson file ----

In [67]:
geometry = dfTGall["geometry"]

In [68]:
#dfgj = dfTG2.drop(["lat", "lon"], axis=1)
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dfTGall, crs=crs, geometry=geometry)

In [69]:
gdf.to_file("TGall.json", driver="GeoJSON")